# Solve *Easy21* by TD(&#955;)

In [1]:
import itertools
import random
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook as tqdm

sns.set()
%matplotlib inline

In [2]:
from plot_utils import draw_heatmap
from easy21 import Easy21, HIT, STICK

In [3]:
env = Easy21()

## Explore policy and stepsize function

In [ ]:
def EpsilonGreedyWithDecay(decay_rate, epsilon0):
    def epsilon_greedy_with_decay(qvalues, visits):
        epsilon = epsilon0 / (1.0 + decay_rate * sum(visits))
        n_actions = len(qvalues)
        probs = [epsilon / n_actions] * n_actions
        #maxidx = qvalues.index(max(qvalues))
        maxidx = np.argmax(qvalues)
        probs[maxidx] += (1 - epsilon)
        return probs
    return epsilon_greedy_with_decay

explore_policy = EpsilonGreedyWithDecay(0.01, 1.0)

In [ ]:
def StepSizeWithDecay(decay_rate, alpha0):
    def step_size_with_decay(visit, *args, **kwargs):
        return alpha0 / (1.0 + decay_rate * visit)
    return step_size_with_decay

stepsize = StepSizeWithDecay(1.0, 1.0)

## Sarsa(&#955;) algorithm

1. Initialize Q function.
2. Set $t=0$ and initialize $s_t$.
3. Set $e(s, a)=0$ for all states and action $s, a$.
4. Choose action $a_t$ by exploration policy to $s_t$.
5. Take $a_t$ and observe $r_{t+1}, s_{t+1}$.
6. Choose action $a_{t+1}$ by exploration policy to $s_{t+1}$.
7. Update $e$ by
$$
e(s) \leftarrow \begin{cases}
  \gamma\lambda e(s, a) + 1  && \text{if } s = s_t \text{ and } a = a_t\\
  \gamma\lambda e(s, a)      && \text{otherwise}
\end{cases}
$$
8. For each $(s, a)$, update Q function by
$$
Q(s, a) \leftarrow Q(s, a) + \alpha \cdot  e(s, a) \left( r_{t+1} + \gamma Q \left( s_{t+1}, a_{t+1} \right) - Q \left( s_t, a_t \right) \right)
$$
9. If $s_{t+1}$ is not terminal, set $t \leftarrow t+1$ and go to 5.
10. Finish if maximum iteration has reached, otherwise go to 2.

Remarks

* $e(s, a)$ is called the *eligibility trace* for state-action pairs. It tracks the contribution of $(s, a)$ to the current value.
* The Q function is update not only for the current observation, but for all state-actions every period. In the implementation, we do this by vectorized operations using `numpy`.

In [ ]:
def sarsa_lambda_episode(qs, ns, env, policy, stepsize, gamma, lambda_):
    """
    Run an episode for Sarsa(lambda) control.
    
    qs and ns are array of shape (s1, s2, ..., a). Where s1, s2,... are state variables and a is action.
    We assume that state is a tuple of nonnegative integers, and action is an nonnegative integer, and
    use them as indices as-is.
    """
    s = env.reset()
    es = np.zeros_like(qs)
    
    prob = policy(qs[s], ns[s])
    a = random.choices(range(env.num_actions), prob)[0]
    while True:
        s1, r, done, _ = env.step(a)
        alpha = stepsize(ns[(*s, a)])
        ns[(*s, a)] += 1.0
        
        es *= (gamma*lambda_)
        es[(*s, a)] += 1.0
        if done:
            delta = r - qs[(*s, a)]
            qs += alpha * es * delta
            break
        else:
            prob = policy(qs[s1], ns[s1])
            a1 = random.choices(range(env.num_actions), prob)[0]
            delta = r + gamma * qs[(*s1, a1)] - qs[(*s, a)]
            qs += alpha * es * delta
            s, a = s1, a1

In [ ]:
# We add index 0, which is not used by the algorithm.
# By doing so states match the indices exactly.
qs = np.zeros((22, 11, 2))
ns = np.zeros((22, 11, 2))

In [ ]:
for i in tqdm(range(10**6)):
    sarsa_lambda_episode(qs, ns, env, explore_policy, stepsize, 1.0, 0.5)

In [ ]:
def to_state_value(qs):
    return np.max(qs, axis=-1)

def to_policy(qs):
    return np.argmax(qs, axis=-1)

In [ ]:
V = to_state_value(qs)

fig, ax = plt.subplots(figsize=(6, 6))
draw_heatmap(V[1:, 1:], ax=ax, title="State Value")
fig.tight_layout()

q_diff = qs[:, :, 1] - qs[:, :, 0]
policy = to_policy(qs)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))
draw_heatmap(q_diff[1:, 1:], ax=ax1, title="Q(hit) - Q(stick)")
draw_heatmap(policy[1:, 1:], ax=ax2, title="Policy(Hit = 1, Stick = 0)", fmt="d")
fig.tight_layout()
None

## Compare to the DP result

In [ ]:
f = np.load("result/dp.npz")
Q_dp = f["Q"]
Q_dp.shape

In [ ]:
maes = {}
for lambda_ in np.arange(0.0, 1.1, 0.1):
    lambda_ = np.round(lambda_, decimals=1)  # to avoid small deviation like 0.30000000000000004
    clear_output(wait=True)
    print("lambda =", lambda_)
    qs = np.zeros((22, 11, 2))
    ns = np.zeros((22, 11, 2))
    mae = []
    for i in tqdm(range(10**7)):
        sarsa_lambda_episode(qs, ns, env, explore_policy, stepsize, 1.0, lambda_)
        if (i + 1) % 1000 == 0:
            mae.append(np.mean(np.abs(qs[1:, 1:] - Q_dp)))
            print("\rIter #%d, MAE = %.6f" % (i+1, mae[-1]), end="")
    maes[lambda_] = mae

In [ ]:
mae_df = pd.DataFrame(maes)

fig, ax = plt.subplots(figsize=(8, 5))
mae_df.plot(ax=ax)
ax.set_xlabel("Iteration")
ax.set_ylabel("Mean absolute error")
ax.set_yscale("log")

In [ ]:
iters = np.linspace(0, len(mae_df)-1, num=6, dtype=int)
fig, axes = plt.subplots(3, 2, figsize=(12, 8), sharex=True, sharey=False)
for i, j in itertools.product(range(3), range(2)):
    k = iters[2*i + j]
    it = 1000 * (k + 1)
    ax = axes[i][j]
    ax.plot(mae_df.iloc[k], marker="o")
    ax.set_xlabel("lambda")
    ax.set_ylabel("Mean absolute error")
    ax.set_title("Iter = %d" % it)
fig.tight_layout()
None

## Off-policy TD(&#955;)

We will implement two variants of off-policy TD(&#955;) algorithms, Watkin's and Na&#239;ve Q(&#955;).
When extending Q-learning with eligibility traces, consideration taken is the treatment of exploratory actions.
* **Watkin's Q(&#955;)**: Zero-out all eligibility traces when non-greedy action is taken.
* **Na&#239;ve Q(&#955;)**: Keep all traces as for Sarasa.

## Watkins's Q(&#955;)

1. Initialize Q function.
2. Set $t=0$ and initialize $s_t$.
3. Set $e(s, a)=0$ for all states and actions $s, a$.
4. Choose action $a_t$ by exploration policy to $s_t$.
5. Take $a_t$ and observe $r_{t+1}, s_{t+1}$.
6. Choose action $a_{t+1}$ by exploration policy to $s_{t+1}$.
7. If $Q(s_{t+1}, a_{t+1}) \not= \max_a Q(s_{t+1}, a)$, set $e(s, a) = 0$ for all $s, a$.
8. Update $e$ by
$$
e(s) \leftarrow \begin{cases}
  \gamma\lambda e(s, a) + 1  && \text{if } s = s_t \text{ and } a = a_t\\
  \gamma\lambda e(s, a)      && \text{otherwise}
\end{cases}
$$
9. For each $(s, a)$, update Q function by
$$
Q(s, a) \leftarrow Q(s, a) + \alpha \cdot  e(s, a) \left( r_{t+1} + \gamma \max_a Q \left( s_{t+1}, a \right) - Q \left( s_t, a_t \right) \right)
$$
10. If $s_{t+1}$ is not terminal, set $t \leftarrow t+1$ and go to 5.
10. Finish if maximum iteration has reached, otherwise go to 2.


## Na&#239;ve Q(&#955;)

1. Initialize Q function.
2. Set $t=0$ and initialize $s_t$.
3. Set $e(s, a)=0$ for all states and actions $s, a$.
4. Choose action $a_t$ by exploration policy to $s_t$.
5. Take $a_t$ and observe $r_{t+1}, s_{t+1}$.
6. Update $e$ by
$$
e(s) \leftarrow \begin{cases}
  \gamma\lambda e(s, a) + 1  && \text{if } s = s_t \text{ and } a = a_t\\
  \gamma\lambda e(s, a)      && \text{otherwise}
\end{cases}
$$

7. For each $(s, a)$, update Q function by
$$
Q(s, a) \leftarrow Q(s, a) + \alpha \cdot  e(s, a) \left( r_{t+1} + \gamma \max_a Q \left( s_{t+1}, a \right) - Q \left( s_t, a_t \right) \right)
$$
8. If $s_{t+1}$ is not terminal, set $t \leftarrow t+1$ and go to 4.
9. Finish if maximum iteration has reached, otherwise go to 2.

In [ ]:
def q_lambda_episode(qs, ns, env, policy, stepsize, gamma, lambda_, watkins=True):
    """
    Run an episode for Q(lambda) control.
    
    qs and ns are array of shape (s1, s2, ..., a). Where s1, s2,... are state variables and a is action.
    We assume that state is a tuple of nonnegative integers, and action is an nonnegative integer, and
    use them as indices as-is.
    """
    s = env.reset()
    es = np.zeros_like(qs)
    
    prob = policy(qs[s], ns[s])
    while True:
        a = random.choices(range(env.num_actions), prob)[0]
        s1, r, done, _ = env.step(a)
        alpha = stepsize(ns[(*s, a)])
        ns[(*s, a)] += 1.0
        
        if watkins and qs[(*s, a)] != max(qs[s]):
            es[:] = 0.0
        es[(*s, a)] += 1.0
        
        if done:
            delta = r - qs[(*s, a)]
            qs += alpha * es * delta
            break
        else:
            prob = policy(qs[s1], ns[s1])
            delta = r + gamma * max(qs[s1]) - qs[(*s, a)]
            qs += alpha * es * delta
            es *= (gamma*lambda_)            
            s = s1

In [ ]:
# Watkins's Q
qs = np.zeros((22, 11, 2))
ns = np.zeros((22, 11, 2))

for i in tqdm(range(10**6)):
    q_lambda_episode(qs, ns, env, explore_policy, stepsize, 1.0, 0.5, watkins=True)


V = to_state_value(qs)
fig, ax = plt.subplots(figsize=(6, 6))
draw_heatmap(V[1:, 1:], ax=ax, title="State Value")
fig.tight_layout()

q_diff = qs[:, :, 1] - qs[:, :, 0]
policy = to_policy(qs)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))
draw_heatmap(q_diff[1:, 1:], ax=ax1, title="Q(hit) - Q(stick)")
draw_heatmap(policy[1:, 1:], ax=ax2, title="Policy(Hit = 1, Stick = 0)", fmt="d")
fig.tight_layout()
None

In [ ]:
# Naive Q
qs = np.zeros((22, 11, 2))
ns = np.zeros((22, 11, 2))

for i in tqdm(range(10**6)):
    q_lambda_episode(qs, ns, env, explore_policy, stepsize, 1.0, 0.5, watkins=False)


V = to_state_value(qs)
fig, ax = plt.subplots(figsize=(6, 6))
draw_heatmap(V[1:, 1:], ax=ax, title="State Value")
fig.tight_layout()

q_diff = qs[:, :, 1] - qs[:, :, 0]
policy = to_policy(qs)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 6))
draw_heatmap(q_diff[1:, 1:], ax=ax1, title="Q(hit) - Q(stick)")
draw_heatmap(policy[1:, 1:], ax=ax2, title="Policy(Hit = 1, Stick = 0)", fmt="d")
fig.tight_layout()
None

*Remark: Both Q(&#955;) methods seem to be working well, although their convergence property is theoretically unknown.*

## Compare to the DP result

In [ ]:
maes = {}
for lambda_, watkins in itertools.product(np.arange(0.0, 1.1, 0.1), [True, False]):
    lambda_ = np.round(lambda_, decimals=1)  # to avoid small deviation like 0.30000000000000004
    clear_output(wait=True)
    print("lambda, watkins =", lambda_, watkins)
    qs = np.zeros((22, 11, 2))
    ns = np.zeros((22, 11, 2))
    mae = []
    for i in tqdm(range(10**7)):
        sarsa_lambda_episode(qs, ns, env, explore_policy, stepsize, 1.0, lambda_)
        if (i + 1) % 1000 == 0:
            mae.append(np.mean(np.abs(qs[1:, 1:] - Q_dp)))
            print("\rIter #%d, MAE = %.6f" % (i+1, mae[-1]), end="")
    maes[(watkins, lambda_)] = mae

In [ ]:
mae_df = pd.DataFrame(maes)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))

mae_df[True].plot(ax=ax1)
ax1.set_xlabel("Iteration")
ax1.set_ylabel("Mean absolute error")
ax1.set_yscale("log")
ax1.set_title("Watkins's Q($\lambda$)")

mae_df[False].plot(ax=ax2)
ax2.set_xlabel("Iteration")
ax2.set_ylabel("Mean absolute error")
ax2.set_yscale("log")
ax2.set_title("Naive Q($\lambda$)")
fig.tight_layout()
None

In [ ]:
iters = np.linspace(0, len(mae_df)-1, num=6, dtype=int)
fig, axes = plt.subplots(3, 2, figsize=(12, 8), sharex=True, sharey=False)
for i, j in itertools.product(range(3), range(2)):
    k = iters[2*i + j]
    it = 1000 * (k + 1)
    ax = axes[i][j]
    ax.plot(mae_df[True].iloc[k], marker="o", label="Watkins Q($\lambda$)")
    ax.plot(mae_df[False].iloc[k], marker="o", label="Naive Q($\lambda$)")
    ax.set_xlabel("lambda")
    ax.set_ylabel("Mean absolute error")
    ax.set_title("Iter = %d" % it)
    ax.legend()
fig.tight_layout()
None